### **Requirements**

In [7]:
!pip install qiskit_nature
!pip install pyscf
!pip install rdkit
!pip install qiskit_aer
!pip install openfermion openfermionpyscf
!pip install scikit-learn
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.0 MB/s eta 0:00:00


### **Data Preparation**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
url = 'https://raw.githubusercontent.com/TastelessCandyzzz/Quantum_Chem_Hackathon/main/bace.csv'

In [60]:
import pandas as pd
df = pd.read_csv(url,usecols=["mol","Class","MW"])
print(df.head())

                                                 mol  Class         MW
0  O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2c...      1  431.56979
1  Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(...      1  657.81073
2  S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...      1  591.74091
3  S1(=O)(=O)C[C@@H](Cc2cc(O[C@H](COCC)C(F)(F)F)c...      1  591.67828
4  S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...      1  629.71283


In [61]:
df=df.loc[df["MW"]<320]
del df["MW"]
df.reset_index(drop=True, inplace=True)
df.shape

(81, 2)

In [62]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np

def smiles_to_fingerprint(smiles, radius=2, n_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return np.array(fp)
    else:
        return np.zeros((n_bits,))

# Apply to all molecules
df['features'] = df['mol'].apply(smiles_to_fingerprint)

[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerator
[17:49:09] DEPRECATION WARNING: please use MorganGenerat

In [63]:
X = np.stack(df["features"])  # Convert feature column into a NumPy matrix
y = df['Class']

## **Calculating Qubit Hamiltonian, VQE ground-state energy, Hartree–Fock energy, Correlation energy, HOMO LUMO Gap and Dipole Moment**

In [8]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolops, Descriptors

# --- PySCF Imports ---
from pyscf import gto, scf

# --- Qiskit Core Imports ---
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import efficient_su2
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import BackendEstimatorV2

# --- Qiskit Nature Imports ---
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer

# ============================================================
# Helper: Generate 3D geometry from an RDKit Mol object
# ============================================================
def mol_to_geometry(mol: Chem.Mol):
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(mol)
    atoms = mol.GetAtoms()
    conf = mol.GetConformer()
    return "; ".join(
        f"{atom.GetSymbol()} "
        f"{conf.GetAtomPosition(atom.GetIdx()).x:.4f} "
        f"{conf.GetAtomPosition(atom.GetIdx()).y:.4f} "
        f"{conf.GetAtomPosition(atom.GetIdx()).z:.4f}"
        for atom in atoms
    )

# ============================================================
# Step 1: Convert geometry → qubit Hamiltonian
# ============================================================
def molecule_to_qubit_hamiltonian(
    geometry,
    basis="sto3g",
    charge=0,
    spin_multiplicity=1,
    active_electrons=2,
    active_orbitals=2,
):
    driver = PySCFDriver(
        atom=geometry, basis=basis, charge=charge,
        spin=(spin_multiplicity - 1), unit=DistanceUnit.ANGSTROM
    )
    es_problem = driver.run()
    freezer = FreezeCoreTransformer(freeze_core=True)
    transformer = ActiveSpaceTransformer(
        num_electrons=active_electrons, num_spatial_orbitals=active_orbitals
    )
    es_problem = freezer.transform(es_problem)
    es_problem = transformer.transform(es_problem)
    mapper = JordanWignerMapper()
    second_q_hamiltonian = es_problem.second_q_ops()[0]
    return {"qubit_operator": mapper.map(second_q_hamiltonian), "es_problem": es_problem}

# ============================================================
# Step 2: Main function to compute all descriptors
# ============================================================
def compute_quantum_descriptors(
    smiles: str,
    basis: str = "sto3g",
    active_electrons: int or tuple = 2,
    active_orbitals: int = 2,
):
    mol_rdkit = Chem.MolFromSmiles(smiles)
    if mol_rdkit is None:
        raise ValueError("Invalid SMILES string provided.")

    charge = rdmolops.GetFormalCharge(mol_rdkit)
    num_radical_electrons = Descriptors.NumRadicalElectrons(mol_rdkit)
    spin_multiplicity = num_radical_electrons + 1

    print(f"\n[•] Processing molecule: {smiles}")
    print(f"[i] Auto-detected Charge: {charge}, Auto-detected Spin: {spin_multiplicity}")

    geometry = mol_to_geometry(mol_rdkit)
    out = molecule_to_qubit_hamiltonian(
        geometry, basis, charge, spin_multiplicity, active_electrons, active_orbitals
    )
    qubit_op, es_problem = out["qubit_operator"], out["es_problem"]
    hf_energy = es_problem.reference_energy
    print(f"[+] Hartree–Fock energy: {hf_energy:.6f} Hartree")

    backend_sim = AerSimulator()
    estimator = BackendEstimatorV2(backend=backend_sim)
    ansatz = efficient_su2(qubit_op.num_qubits, reps=2, entanglement="linear")
    ansatz = transpile(ansatz, backend=backend_sim)
    optimizer = COBYLA(maxiter=200)
    vqe_solver = VQE(estimator, ansatz, optimizer)
    vqe_result = vqe_solver.compute_minimum_eigenvalue(qubit_op)
    vqe_energy = vqe_result.eigenvalue.real
    print(f"[+] VQE ground-state energy: {vqe_energy:.6f} Hartree")

    # --- PySCF calculations for additional properties ---
    mol_pyscf = gto.Mole(
        atom=geometry, basis=basis, charge=charge, spin=spin_multiplicity - 1
    ).build()
    mf = scf.RHF(mol_pyscf).run() if spin_multiplicity == 1 else scf.UHF(mol_pyscf).run()

    # Using the robust, low-level dipole calculation
    dm = mf.make_rdm1()
    dip_ints = mf.mol.intor('int1e_r', comp=3)
    elec_dip = -np.einsum('xij,ji->x', dip_ints, dm)
    nuc_dip = np.einsum('i,ix->x', mf.mol.atom_charges(), mf.mol.atom_coords())
    dipole_vector = elec_dip + nuc_dip
    dipole_moment = np.linalg.norm(dipole_vector)

    # --- Property Extraction ---
    if es_problem.orbital_energies_b is None:
        num_occupied = es_problem.num_alpha
        homo_energy = es_problem.orbital_energies[num_occupied - 1]
        lumo_energy = es_problem.orbital_energies[num_occupied]
    else:
        alpha_homo = es_problem.orbital_energies[es_problem.num_alpha - 1]
        beta_homo = es_problem.orbital_energies_b[es_problem.num_beta - 1]
        homo_energy = max(alpha_homo, beta_homo)
        lumo_energy = es_problem.orbital_energies[es_problem.num_alpha]

    # --- Collect all descriptors ---
    return {
        "smiles": smiles,
        "charge": charge,
        "spin_multiplicity": spin_multiplicity,
        "hf_energy": hf_energy,
        "vqe_energy": vqe_energy,
        "correlation_energy": hf_energy - vqe_energy,
        "n_qubits": qubit_op.num_qubits,
        "homo_energy": homo_energy,
        "lumo_energy": lumo_energy,
        "homo_lumo_gap": lumo_energy - homo_energy,
        "dipole_moment": dipole_moment,
    }

# ============================================================
# Example Run
# ============================================================
if __name__ == "__main__":
    hydroxide_results = compute_quantum_descriptors(smiles="[OH-]")
    print("\n--- Final Descriptors for [OH-] ---")
    for key, value in hydroxide_results.items():
        print(f"{key:<20}: {value:.6f}" if isinstance(value, float) else f"{key:<20}: {value}")
    print("-----------------------------------")


[•] Processing molecule: [OH-]
[i] Auto-detected Charge: -1, Auto-detected Spin: 1
[+] Hartree–Fock energy: -74.060416 Hartree
[+] VQE ground-state energy: -0.299934 Hartree
converged SCF energy = -74.0604157013592

--- Final Descriptors for [OH-] ---
smiles              : [OH-]
charge              : -1
spin_multiplicity   : 1
hf_energy           : -74.060416
vqe_energy          : -0.299934
correlation_energy  : -73.760481
n_qubits            : 4
homo_energy         : 0.247616
lumo_energy         : 1.222037
homo_lumo_gap       : 0.974422
dipole_moment       : 0.804211
-----------------------------------


In [12]:
df.to_csv('/content/drive/MyDrive/BACE_.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time # Used for the dummy function
from google.colab import files

output_csv_path = '/content/drive/MyDrive/BACE_with_quantum_features.csv'
input_csv_path = '/content/drive/MyDrive/BACE_.csv'
save_interval = 8

# List of new columns we will be adding
quantum_cols = [
    "hf_energy", "vqe_energy", "correlation_energy", "n_qubits",
    "homo_energy", "lumo_energy", "homo_lumo_gap", "dipole_moment"
]

# Check if an output file already exists to resume from it
if os.path.exists(output_csv_path):
    print(f"Resuming from existing file: '{output_csv_path}'")
    # When reading, set index_col=0 to prevent creating a new 'Unnamed: 0' column
    df = pd.read_csv(output_csv_path, index_col=0)
else:
    print(f"Starting a new run. Loading data from '{input_csv_path}'")
    df = pd.read_csv(input_csv_path)
    for col in quantum_cols:
        df[col] = np.nan

# We iterate through the DataFrame's index
for i in tqdm(df.index, desc="Computing quantum descriptors"):
    # Check if the calculation for this row has already been done
    # We only need to check one column; if it's not NaN, we assume the row is complete.
    if pd.notna(df.loc[i, 'hf_energy']):
        continue # Skip to the next molecule

    # Get the SMILES string for the current molecule
    smiles = df.loc[i, 'mol']

    try:
        descriptors = compute_quantum_descriptors(smiles, active_electrons=2, active_orbitals=2)

        if descriptors:
            # If successful, update all columns for the current index 'i'
            for key, value in descriptors.items():
                df.loc[i, key] = value
        else:
            df.loc[i, 'hf_energy'] = 'Failed'

    except Exception as e:
        print(f"[!] Error processing molecule {i} ({smiles}): {e}")
        # Mark with an 'Error' message to avoid re-calculating this row next time
        df.loc[i, 'hf_energy'] = 'Error'


    if (i + 1) % save_interval == 0 or (i + 1) == len(df):
        print(f"\n Processed up to molecule {i + 1}. Saving progress to '{output_csv_path}'...")
        df.to_csv('/content/drive/MyDrive/BACE_with_quantum_features.csv', index=False)

print("\n All molecules processed. Final data saved successfully!")
print(df.head())

Starting a new run. Loading data from '/content/drive/MyDrive/BACE_.csv'


Computing quantum descriptors:   0%|          | 0/81 [00:00<?, ?it/s]


[•] Processing molecule: O=C1N(C)C(N[C@@]1(CC1CCCCC1)CCC1CCCCC1)=N
[i] Auto-detected Charge: 0, Auto-detected Spin: 1
[+] Hartree–Fock energy: -965.086237 Hartree
[+] VQE ground-state energy: -0.861831 Hartree
converged SCF energy = -965.086236582539


Computing quantum descriptors:   1%|          | 1/81 [03:12<4:16:19, 192.24s/it]


[•] Processing molecule: O=C(NC(=[NH2+])N)Cn1c(ccc1-c1ccccc1)-c1ccccc1
[i] Auto-detected Charge: 1, Auto-detected Spin: 1
[+] Hartree–Fock energy: -1010.330855 Hartree
[+] VQE ground-state energy: -0.752921 Hartree
converged SCF energy = -1010.33085493198


...


 Processed up to molecule 48. Saving progress to '/content/drive/MyDrive/BACE_with_quantum_features.csv'...

[•] Processing molecule: O1CC(N=C(N)C1)(C)c1cc(ccc1)-c1cc(OC)ccc1
[i] Auto-detected Charge: 0, Auto-detected Spin: 1
[+] Hartree–Fock energy: -940.066574 Hartree
[+] VQE ground-state energy: -0.500651 Hartree
converged SCF energy = -940.066573599079


Computing quantum descriptors:  60%|██████    | 49/81 [1:34:44<1:11:45, 134.53s/it]


[•] Processing molecule: O=C1N(C)C(=NC1(c1ccccc1)c1ccccc1)N
[i] Auto-detected Charge: 0, Auto-detected Spin: 1
[+] Hartree–Fock energy: -842.218371 Hartree
[+] VQE ground-state energy: -0.692902 Hartree
converged SCF energy = -842.218370880246


Computing quantum descriptors:  62%|██████▏   | 50/81 [1:36:36<1:06:01, 127.80s/it]


[•] Processing molecule: O(C)c1cc(ccc1)C1(N=C(N)N(C)C1=O)c1ccccc1
[i] Auto-detected Charge: 0, Auto-detected Spin: 1
[+] Hartree–Fock energy: -954.629590 Hartree
[+] VQE ground-state energy: -0.765461 Hartree


## **Training ML Model Using Classically Obtained Features Only**

In [64]:
import pandas as pd
import numpy as np

d = pd.read_csv("/content/drive/MyDrive/BACE_with_quantum_features.csv")
print(d.head())

                                               mol  Class           features  \
0        O=C1N(C)C(N[C@@]1(CC1CCCCC1)CCC1CCCCC1)=N      0  [0 0 1 ... 0 0 0]   
1    O=C(NC(=[NH2+])N)Cn1c(ccc1-c1ccccc1)-c1ccccc1      0  [0 0 0 ... 0 0 0]   
2   Clc1cc(ccc1)-c1cc(ccc1)[C@@]1(OCCC(=[NH+]1)N)C      0  [0 0 0 ... 0 0 0]   
3         [nH]1c2cc(ccc2cc1)-c1cc(ccc1)CNc1cccnc1N      0  [0 0 0 ... 0 0 0]   
4  O=C1N(C)C(N[C@](C1)(C)[C@@H]1C[C@H]1c1ccccc1)=N      0  [0 0 0 ... 0 0 0]   

     hf_energy  vqe_energy  correlation_energy  n_qubits  homo_energy  \
0  -965.086237   -0.861831         -964.224406       4.0    -0.245345   
1 -1010.330855   -0.752921        -1009.577934       4.0    -0.283762   
2 -1282.140915   -0.901309        -1281.239606       4.0    -0.348090   
3  -973.453334   -0.614258         -972.839076       4.0    -0.197059   
4  -807.135855   -0.734406         -806.401449       4.0    -0.242222   

   lumo_energy  homo_lumo_gap  dipole_moment  \
0     0.281297       0.526642   

In [65]:
d["feature"]=df["features"]
d["feature"]

,feature
0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
76,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
77,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
78,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
79,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


In [66]:
d.dropna(inplace=True)
d.reset_index(drop=True, inplace=True)

In [67]:
import numpy as np

X = np.stack(d["feature"])  # Convert feature column into a NumPy matrix
y = d['Class']
X.dtype

dtype('int64')

In [72]:
y.value_counts()

,count
Class,
0,39
1,9


In [68]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
pca.fit(X)
X_pca = pca.transform(X)
print("X_train shape after PCA:", X_pca.shape)

X_train shape after PCA: (48, 35)


In [74]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=1)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (38, 35)
X_test shape: (10, 35)
y_train shape: (38,)
y_test shape: (10,)


In [75]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Initialize and train the XGBoost Classifier
xgb_model_classical = xgb.XGBClassifier(objective='binary:logistic', random_state=34, scale_pos_weight=8)
xgb_model_classical.fit(X_train, y_train)

print("XGBoost model (classical features only) trained successfully!")

# Make predictions on the test set
y_pred_xgb_classical = xgb_model_classical.predict(X_test)

# Evaluate the XGBoost model
accuracy_xgb_classical = accuracy_score(y_test, y_pred_xgb_classical)
f1_xgb_classical = f1_score(y_test, y_pred_xgb_classical)
precision_xgb_classical = precision_score(y_test, y_pred_xgb_classical)
recall_xgb_classical = recall_score(y_test, y_pred_xgb_classical)

# Print the evaluation metrics for XGBoost (classical features only)
print("\nXGBoost Model (Classical Features Only) Evaluation:")
print(f"Accuracy: {accuracy_xgb_classical:.4f}")
print(f"F1 Score: {f1_xgb_classical:.4f}")
print(f"Precision: {precision_xgb_classical:.4f}")
print(f"Recall: {recall_xgb_classical:.4f}")

XGBoost model (classical features only) trained successfully!

XGBoost Model (Classical Features Only) Evaluation:
Accuracy: 0.9000
F1 Score: 0.8000
Precision: 1.0000
Recall: 0.6667


## **Training ML Model Using the Quantum Features Also**

In [79]:
X_=np.array(pd.DataFrame(d,columns=["hf_energy","vqe_energy","homo_energy","lumo_energy","dipole_moment"]))
X_Q=np.hstack((X_pca,X_))

from sklearn.model_selection import train_test_split
X_train_Q, X_test_Q, y_train_Q, y_test_Q = train_test_split(X_Q, y, test_size=0.2, random_state=2)

print("X_train shape:", X_train_Q.shape)
print("X_test shape:", X_test_Q.shape)

X_train shape: (38, 40)
X_test shape: (10, 40)


In [84]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Initialize and train the XGBoost Classifier
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=4, scale_pos_weight=8)
xgb_model.fit(X_train_Q, y_train_Q)

print("XGBoost model trained successfully!")

# Make predictions on the test set
y_pred_xgb = xgb_model.predict(X_test_Q)

# Evaluate the XGBoost model
accuracy_xgb = accuracy_score(y_test_Q, y_pred_xgb)
f1_xgb = f1_score(y_test_Q, y_pred_xgb)
precision_xgb = precision_score(y_test_Q, y_pred_xgb)
recall_xgb = recall_score(y_test_Q, y_pred_xgb)

# Print the evaluation metrics for XGBoost
print("\nXGBoost Model Evaluation:")
print(f"Accuracy: {accuracy_xgb:.4f}")
print(f"F1 Score: {f1_xgb:.4f}")
print(f"Precision: {precision_xgb:.4f}")
print(f"Recall: {recall_xgb:.4f}")

XGBoost model trained successfully!

XGBoost Model Evaluation:
Accuracy: 0.9000
F1 Score: 0.6667
Precision: 0.5000
Recall: 1.0000
